# 1) **Data Collection**

## 1.1) Occurence data

In [ ]:
import requests
import pandas as pd

# Define the target species by its scientific name (Wild Yak only)
species_name = "Bos mutus"

# Base URL endpoints for the GBIF API
species_api = "https://api.gbif.org/v1/species/match?name="
occurrence_api = "https://api.gbif.org/v1/occurrence/search"

# Initialize a container to store all occurrence records
all_records = []

# Send a GET request to retrieve the GBIF species key (usageKey) using the scientific name
resp = requests.get(species_api + requests.utils.requote_uri(species_name))
data = resp.json()
if "usageKey" in data:
    species_key = data["usageKey"]
    print(f"GBIF species key for {species_name}: {species_key}")
else:
    raise ValueError(f"Species key not found for {species_name}")

# Paginate through occurrence records, with a maximum of 300 records per page
offset = 0
while True:
    params = {
        "taxonKey": species_key,
        "hasCoordinate": "true",
        "limit": 300,
        "offset": offset
        # Note: Filters for year and occurrenceStatus have been removed to maximize results
    }
    res = requests.get(occurrence_api, params=params)
    res.raise_for_status()
    results = res.json()

    # Exit the loop if no results are returned or if there are no more records
    if "results" not in results or not results["results"]:
        break

    # Process each occurrence record and extract the relevant fields
    for rec in results["results"]:
        lat = rec.get("decimalLatitude")
        lon = rec.get("decimalLongitude")
        date = rec.get("eventDate") or rec.get("year")
        source = rec.get("datasetName")

        # Only include records that have both latitude and longitude information
        if lat is not None and lon is not None:
            all_records.append({
                "scientificName": species_name,
                "latitude": lat,
                "longitude": lon,
                "date": date,
                "source": source
            })

    offset += 300
    if results.get("endOfRecords"):
        break

# Create a DataFrame from the collected records
df = pd.DataFrame(all_records)

# Remove duplicate records based on latitude, longitude, and date
df.drop_duplicates(subset=["latitude", "longitude", "date"], inplace=True)

# Remove any records that have missing latitude or longitude values
df.dropna(subset=["latitude", "longitude"], inplace=True)

# Export the cleaned data to a CSV file
df.to_csv("wild_yak_occurrences.csv", index=False)
print(f"Saved {len(df)} Wild Yak occurrence records to wild_yak_occurrences.csv")


## 1.2) Weather Data 

In [ ]:
import os
import requests
import zipfile

# Define the output directory where all climate data will be stored.
output_dir = "climate_data"                  # Base folder for all data

# Define the variables to download:
# ppt: precipitation, tmin: minimum temperature, tmax: maximum temperature.
variables = ["ppt", "tmin", "tmax"]

# Define the range of years to download.
# Note: The comment states "years 1990 through 2020 inclusive" but the code uses years 2009 to 2024.
years = range(2009, 2025)

# TerraClimate file URL template (provides NetCDF files)
base_url = "http://thredds.northwestknowledge.net:8080/thredds/fileServer/TERRACLIMATE_ALL/data"

# Create the base output directory if it does not already exist.
os.makedirs(output_dir, exist_ok=True)

# Process each climate variable.
for var in variables:
    # Define the zip file name for the current variable.
    zip_filename = os.path.join(output_dir, f"{var}_1990_2020.zip")

    # Create a new zip file for this variable.
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Process each year in the specified range.
        for year in years:
            # Construct the file name for the TerraClimate data file.
            filename = f"TerraClimate_{var}_{year}.nc"

            # Define a temporary file path for downloading the file.
            temp_path = os.path.join(output_dir, filename)

            # Skip downloading if the file is already added to the ZIP archive.
            if filename in zipf.namelist():
                print(f"{filename} already in ZIP, skipping.")
                continue

            # Construct the full file URL for downloading.
            file_url = f"{base_url}/TerraClimate_{var}_{year}.nc"
            print(f"Downloading {filename} ...")

            # Download the file using streaming mode to handle large files.
            response = requests.get(file_url, stream=True)
            if response.status_code == 200:
                # Write the downloaded content in chunks to the temporary file.
                with open(temp_path, "wb") as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                print(f"Downloaded {filename}")

                # Add the downloaded file to the ZIP archive.
                zipf.write(temp_path, arcname=filename)
                print(f"Added {filename} to ZIP archive.")

                # Remove the temporary file after adding it to the ZIP archive.
                os.remove(temp_path)
            else:
                print(f"Failed to download {file_url} (status code {response.status_code})")

    print(f"All data for {var} saved to {zip_filename}")


## 1.3) Elevation Data

In [ ]:
import os
import glob

# Specify the input folder path where the exported Earth Engine files are located in Google Drive
input_folder = '/content/drive/MyDrive/EarthEngineExports'

# Search for all files with a .tif extension in the input folder
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))

# Print out the total number of TIF files found
print(f"Found {len(tif_files)} TIF files")


In [ ]:
import rasterio
from rasterio.merge import merge
import matplotlib.pyplot as plt

# Open each TIF file and store the opened raster file objects in a list.
src_files_to_mosaic = [rasterio.open(fp) for fp in tif_files]

# Merge all the raster files into one mosaic.
mosaic, out_trans = merge(src_files_to_mosaic)

# Copy metadata from the first raster file and update it with new properties from the merged mosaic.
out_meta = src_files_to_mosaic[0].meta.copy()
out_meta.update({
    "driver": "GTiff",              # Use GeoTIFF format for the output.
    "height": mosaic.shape[1],      # Set the output height based on the mosaic.
    "width": mosaic.shape[2],       # Set the output width based on the mosaic.
    "transform": out_trans,         # Update the transform information.
    "count": 1                      # Specify that there is only one band.
})

# Define the output path for the merged raster.
output_path = '/content/drive/MyDrive/merged_asia_elevation.tif'

# Write the mosaic to a new GeoTIFF file using the updated metadata.
with rasterio.open(output_path, "w", **out_meta) as dest:
    dest.write(mosaic)

# Print a confirmation message indicating that the merged raster was saved.
print(f"Merged raster saved to: {output_path}")


In [ ]:
from osgeo import gdal
gdal.UseExceptions()
import glob
import os

# Step 1: Build a virtual raster (VRT) that references all the individual elevation tiles.
tiles_dir = r"C:\\Users\\FENIL\\Downloads\\Elevation"
tif_files = glob.glob(os.path.join(tiles_dir, "*.tif"))

# Create a VRT file that aggregates all the TIFF files from the specified directory.
vrt_path = "asia_elevation.vrt"
gdal.BuildVRT(vrt_path, tif_files)

# Step 2: Convert the VRT into a single GeoTIFF.
# Define the bounding box for the Asia region.
# Note: gdal.Translate requires the window in the following order:
# [min_x (left), max_y (top), max_x (right), min_y (bottom)].
asia_bounds = [25, 60, 150, -10]  # Adjust these boundaries if needed.

output_tif = "asia_elevation_merged.tif"

# Set translation options:
# - TILED=YES creates internal tiling for efficient access.
# - COMPRESS=DEFLATE applies compression to reduce file size.
# - BIGTIFF=YES allows for the creation of files larger than 4GB if necessary.
translate_options = gdal.TranslateOptions(
    projWin=asia_bounds,  # Clip the data to the extent of Asia.
    creationOptions=["TILED=YES", "COMPRESS=DEFLATE", "BIGTIFF=YES"]
)

# Generate the final GeoTIFF by translating the VRT with the specified options.
gdal.Translate(output_tif, vrt_path, options=translate_options)

# Print confirmation that the merged GeoTIFF has been created.
print(f"GeoTIFF created: {output_tif}")


In [ ]:
import os
import rasterio
from rasterio.warp import reproject, Resampling
import xarray as xr

# === USER SETTINGS ===
elevation_path = r"asia_elevation_merged.tif"  # Path to the large original elevation raster
ref_nc_path = r"C:\\Users\\FENIL\\Downloads\\climate_data\\ppt_1990_2020\\TerraClimate_ppt_2009.nc"  # Path to a sample climate NetCDF file (any year works)
resampled_output_path = r"elevation_resampled_to_climate.tif"  # Output path for the resampled elevation raster

print("Step 1: Loading reference climate grid...")
# Open the NetCDF file with xarray to access the climate grid
ref_nc = xr.open_dataset(ref_nc_path)
# Select the first month's data from the 'ppt' variable to use as a reference grid
ref_grid = ref_nc['ppt'].isel(time=0)

# Retrieve the shape (number of rows and columns) and geographic bounds (longitude and latitude extents) of the reference grid
target_height, target_width = ref_grid.shape
min_lon, max_lon = float(ref_grid.lon.min()), float(ref_grid.lon.max())
min_lat, max_lat = float(ref_grid.lat.min()), float(ref_grid.lat.max())

print(f"Climate raster size: {target_height} rows × {target_width} cols")
print(f"Geographic extent: lon {min_lon} to {max_lon}, lat {min_lat} to {max_lat}")

# Create a transform object for the output grid based on the bounds and dimensions of the reference climate grid
target_transform = rasterio.transform.from_bounds(
    min_lon, min_lat, max_lon, max_lat,
    target_width, target_height
)

print("\nStep 2: Opening original elevation raster...")
# Open the original elevation raster
with rasterio.open(elevation_path) as src:
    print(f"Original raster CRS: {src.crs}")
    print(f"Original size: {src.height} rows × {src.width} cols")

    # Prepare output profile by copying the original metadata and updating it for the new grid
    profile = src.profile.copy()
    profile.update({
        "height": target_height,       # Set the number of rows to match the reference grid
        "width": target_width,         # Set the number of columns to match the reference grid
        "transform": target_transform, # Set the new transform based on the reference grid bounds
        "compress": 'lzw',             # Use LZW compression for the output file
        "dtype": 'float32'             # Set the data type of the output file
    })

    print("\nStep 3: Creating output file and reprojecting...")
    # Create the output raster file and perform the reprojection/resampling
    with rasterio.open(resampled_output_path, "w", **profile) as dst:
        reproject(
            source=rasterio.band(src, 1),         # Use the first band of the source
            destination=rasterio.band(dst, 1),      # Write to the first band of the destination
            src_transform=src.transform,            # Original source transform
            src_crs=src.crs,                        # Original source CRS
            dst_transform=target_transform,         # New target transform
            dst_crs=src.crs,                        # Keeping the same CRS as the source
            resampling=Resampling.bilinear           # Use bilinear resampling for smoother results
        )

print(f"\nDone! Resampled elevation saved to:\n{resampled_output_path}")


# 2) SDM for Wild Yak

## 2.1) Data Cleaning and EDA

In [ ]:
import pandas as pd
from dateutil import parser

# Define a custom function to extract the year from a date string, even when the format is inconsistent
def extract_year(date_str):
    try:
        # Try parsing the date using dateutil, which can handle many common date formats
        return parser.parse(str(date_str), fuzzy=True).year
    except:
        try:
            # If parsing fails, check if the string is just a year (e.g., "2015")
            if len(str(date_str).strip()) == 4 and str(date_str).isdigit():
                return int(date_str)
        except:
            pass
        # Return None if the date could not be parsed
        return None

# Load the dataset from a CSV file
df = pd.read_csv('C:\\Users\\FENIL\\Downloads\\final_wild_yak_occurrences.csv')

# Apply the year extraction function to the 'date' column and create a new 'year' column
df['year'] = df['date'].apply(extract_year)

# Remove the original 'date' column if it's no longer needed
df = df.drop(columns=['date'])

# Save the updated DataFrame to a new CSV file
df.to_csv('wild_yak_occurrences_years_cleaned.csv', index=False)

# Print the first 15 rows to verify the results
print(df.head(15))


In [ ]:
import pandas as pd

# Load the dataset from the CSV file.
df = pd.read_csv("wild_yak_Final.csv")

# Exclude records where the 'year' is 1905.
df_cleaned = df[df['year'] != 1905]

# Save the cleaned dataset to a new CSV file.
df_cleaned.to_csv("wild_yak_Final_cleaned.csv", index=False)

# Inform that all records from 1905 have been removed.
print("Removed all records from 1905. New file saved as 'wild_yak_Final_cleaned.csv'")


In [ ]:
import pandas as pd
import random
from geopy.distance import distance
from geopy.point import Point

# Function to apply a random spatial adjustment (jitter) to a point within a specified radius in kilometers.
def jitter_point(lat, lon, max_distance_km):
    # Generate a random direction (bearing) between 0 and 360 degrees.
    bearing = random.uniform(0, 360)
    # Generate a random distance up to the maximum specified distance.
    jitter_distance = random.uniform(0, max_distance_km)
    # Define the original point using its latitude and longitude.
    origin = Point(lat, lon)
    # Calculate the destination point given the random distance and bearing.
    destination = distance(kilometers=jitter_distance).destination(origin, bearing)
    return destination.latitude, destination.longitude

# Function to create multiple jittered records for each original record in a dataset.
def generate_multiple_jittered_records(df, lat_col='latitude', lon_col='longitude', jitter_per_record=10, max_distance_km=1):
    jittered_records = []

    # Iterate over each record in the original DataFrame.
    for _, row in df.iterrows():
        # Generate the specified number of jittered records for the current record.
        for _ in range(jitter_per_record):
            jittered_lat, jittered_lon = jitter_point(row[lat_col], row[lon_col], max_distance_km)
            
            # Create a copy of the original record and update its latitude and longitude with the jittered values.
            new_record = row.copy()
            new_record[lat_col] = jittered_lat
            new_record[lon_col] = jittered_lon
            
            # Add the jittered record to the list.
            jittered_records.append(new_record)
    
    # Create a DataFrame from the jittered records.
    jittered_df = pd.DataFrame(jittered_records)
    # Combine the original and jittered records into a single DataFrame.
    combined_df = pd.concat([df, jittered_df], ignore_index=True)

    return combined_df

# Specify the number of jittered records to create per original record and the maximum jitter distance (km).
jitter_per_record = 10  
max_jitter_distance_km = 5  

# Generate the expanded dataset by combining the original data with its jittered versions.
expanded_df = generate_multiple_jittered_records(
    df, 
    lat_col='latitude', 
    lon_col='longitude', 
    jitter_per_record=jitter_per_record, 
    max_distance_km=max_jitter_distance_km
)

# Save the expanded dataset with jittered records to a CSV file.
expanded_df.to_csv('wild_yak_expanded_10x_jitter.csv', index=False)

# Output the first 15 rows of the expanded dataset and display the number of records before and after expansion.
print(expanded_df.head(15))
print(f"Original count: {len(df)}, Expanded count: {len(expanded_df)}")


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geodatasets

# Load the cleaned dataset containing the occurrence records.
df = pd.read_csv("wild_yak_Final_cleaned.csv")

# Create a list of shapely Point objects from the longitude and latitude columns.
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
# Convert the DataFrame to a GeoDataFrame with the specified coordinate reference system (WGS84).
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Load a simple world map from geodatasets for the background.
world = gpd.read_file(geodatasets.get_path("naturalearth.land"))

# Create a sorted list of unique years found in the dataset.
years = sorted(gdf['year'].dropna().unique())

# Generate a map for each individual year.
for year in years:
    # Filter the GeoDataFrame to include only the records for the current year.
    yearly_data = gdf[gdf['year'] == year]

    # Create a new plot figure with a defined size.
    fig, ax = plt.subplots(figsize=(10, 6))
    # Plot the world map as the base layer.
    world.plot(ax=ax, color='lightgray', edgecolor='black')
    # Plot the wild yak occurrence points for the current year on top of the world map.
    yearly_data.plot(ax=ax, color='red', markersize=20, alpha=0.6)

    # Set the title and axis labels for the plot.
    plt.title(f"Wild Yak Occurrences - {year}")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.tight_layout()
    # Save the resulting plot to a PNG file named for the current year.
    plt.savefig(f"wild_yak_{year}.png")
    # Display the plot.
    plt.show()


## 2.2) Modeling and Evaluations

In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import rasterio
from rasterio.transform import rowcol
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import pickle

# ========== CONFIGURATION ==========
# Define file paths and directories for input and output data.
occurrence_csv = "wild_yak_Final_cleaned.csv"   # CSV containing occurrence records.
elevation_path = "elevation_resampled_to_climate.tif"  # Resampled elevation raster file.
landmask_path = "landmask_asia.tif"               # Landmask raster file for Asia.
climate_root = r"C:\Users\FENIL\Downloads\climate_data"  # Root directory for climate data.
ppt_dir = os.path.join(climate_root, "ppt_1990_2020")
tmin_dir = os.path.join(climate_root, "tmin_1990_2020")
tmax_dir = os.path.join(climate_root, "tmax_1990_2020")
output_dir = "sdm_final"                         # Directory to save final outputs.
os.makedirs(output_dir, exist_ok=True)

# Define the years for which to run the analysis and the future climate scenarios.
selected_years = list(range(2009, 2025))
future_scenarios = {2050: {"SSP585": "2050585", "SSP245": "2050245"}}

# Set a random seed for reproducibility.
np.random.seed(42)

# ========== LOAD OCCURRENCE DATA ==========
print("Loading occurrence records...")
# Read in the occurrence data and keep only records with valid latitude, longitude, and year values.
occurrences = pd.read_csv(occurrence_csv)
occurrences = occurrences[['latitude', 'longitude', 'year']].dropna()
# Filter the occurrences to include only those in the selected years.
occurrences = occurrences[occurrences['year'].isin(selected_years)]

# ========== LOAD RASTER DATA ==========
print("Opening elevation and landmask rasters...")
# Open the resampled elevation raster and read the first band.
with rasterio.open(elevation_path) as elev_src:
    elev = elev_src.read(1)
    transform = elev_src.transform  # Get the spatial transform for coordinate conversion.

# Open the landmask raster, converting the data to boolean values.
with rasterio.open(landmask_path) as lm_src:
    landmask = lm_src.read(1).astype(bool)

# ========== FEATURE COLLECTION ==========
# Initialize lists to hold features and corresponding presence/absence labels.
X_all, y_all = [], []

# Loop through each selected year to extract climate and elevation features.
for year in selected_years:
    print(f"Processing year {year}...")
    # Load climate data for precipitation, minimum temperature, and maximum temperature.
    ppt = xr.open_dataset(os.path.join(ppt_dir, f"TerraClimate_ppt_{year}.nc"))['ppt'].sum(dim='time')
    tmin = xr.open_dataset(os.path.join(tmin_dir, f"TerraClimate_tmin_{year}.nc"))['tmin'].mean(dim='time')
    tmax = xr.open_dataset(os.path.join(tmax_dir, f"TerraClimate_tmax_{year}.nc"))['tmax'].mean(dim='time')

    # Filter occurrence records for the current year (presence data).
    presences = occurrences[occurrences['year'] == year]

    # For each presence point, extract the climate variables and elevation.
    for _, row in presences.iterrows():
        lat, lon = row['latitude'], row['longitude']
        # Convert geographic coordinates to raster row and column indices.
        i, j = rowcol(transform, lon, lat)
        # Skip the record if the point does not fall on land.
        if not landmask[i, j]:
            continue
        feat = [
            float(ppt.sel(lat=lat, lon=lon, method='nearest')),
            float(tmin.sel(lat=lat, lon=lon, method='nearest')),
            float(tmax.sel(lat=lat, lon=lon, method='nearest')),
            elev[i, j]
        ]
        # Only include the record if all features are valid numbers.
        if not np.isnan(feat).any():
            X_all.append(feat)
            y_all.append(1)

    # Generate absence points by randomly sampling locations and extracting features.
    absences, attempts = 0, 0
    lats, lons = ppt.lat.values, ppt.lon.values
    # Try to generate twice as many absence points as there are presence points.
    while absences < len(presences) * 2 and attempts < 10000:
        rand_lat, rand_lon = np.random.choice(lats), np.random.choice(lons)
        attempts += 1
        i, j = rowcol(transform, rand_lon, rand_lat)
        if not landmask[i, j]:
            continue
        feat = [
            float(ppt.sel(lat=rand_lat, lon=rand_lon, method='nearest')),
            float(tmin.sel(lat=rand_lat, lon=rand_lon, method='nearest')),
            float(tmax.sel(lat=rand_lat, lon=rand_lon, method='nearest')),
            elev[i, j]
        ]
        if not np.isnan(feat).any():
            X_all.append(feat)
            y_all.append(0)
            absences += 1

print("Features collected.")

# ========== TRAIN MODEL ==========
print("Training model...")
# Convert the feature and label lists into NumPy arrays.
X, y = np.array(X_all), np.array(y_all)
# Split the data into training and testing subsets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Initialize and train a Random Forest classifier.
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)
# Evaluate model performance using the Area Under the ROC Curve (AUC).
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print(f"Model trained. AUC: {auc:.3f}")

# Save the trained model to a file using pickle.
model_filename = os.path.join(output_dir, "random_forest_model.pkl")
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)
print(f"Model saved to {model_filename}")

# ========== PREDICTION AND PLOTTING ==========
# The prediction and plotting functions should be included here.
# Replace the following placeholder with your actual 'predict_and_plot' function.
def predict_and_plot(year, suffix, label):
    # Placeholder function: implement your prediction and plotting logic here.
    # Typically, this function should load the necessary climate data for the given year,
    # use the trained model to predict species distribution,
    # and generate maps that visualize the predictions.
    print(f"Running predictions and plotting for year {year} with label {label}.")

# ========== RUN PREDICTIONS ==========
# Run predictions and plot results for each selected year.
for year in selected_years:
    predict_and_plot(year, year, str(year))

# Run predictions for future climate scenarios.
for future_year, scenarios in future_scenarios.items():
    for label, suffix in scenarios.items():
        predict_and_plot(future_year, suffix, label)


In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import rasterio
from rasterio.transform import rowcol
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    confusion_matrix,
    classification_report,
    RocCurveDisplay
)
import matplotlib.pyplot as plt
import pickle

# === Configuration ===
# Specify paths for input occurrence data, elevation raster, and landmask.
occurrence_csv = "wild_yak_Final_cleaned.csv"
elevation_path = "elevation_resampled_to_climate.tif"
landmask_path = "landmask_asia.tif"

# Define the root directory for climate data and subdirectories for different variables.
climate_root = r"C:\Users\FENIL\Downloads\climate_data"
ppt_dir = os.path.join(climate_root, "ppt_1990_2020")
tmin_dir = os.path.join(climate_root, "tmin_1990_2020")
tmax_dir = os.path.join(climate_root, "tmax_1990_2020")

# Define the range of years to use for model training and evaluation.
selected_years = list(range(2009, 2025))

# Set a fixed random seed for reproducibility.
np.random.seed(42)

# === Load Occurrence Data ===
print("Loading occurrence data...")
# Read occurrence records and select only the necessary columns (latitude, longitude, year).
occurrences = pd.read_csv(occurrence_csv)
occurrences = occurrences[['latitude', 'longitude', 'year']].dropna()
# Filter records to keep only those that fall within the selected years.
occurrences = occurrences[occurrences['year'].isin(selected_years)]

# === Load Elevation and Landmask ===
print("Loading elevation and landmask data...")
# Open the elevation raster and extract the first band along with its spatial transform.
with rasterio.open(elevation_path) as elev_src:
    elev = elev_src.read(1)
    transform = elev_src.transform

# Open the landmask raster and convert its values to boolean for easier processing.
with rasterio.open(landmask_path) as lm_src:
    landmask = lm_src.read(1).astype(bool)

# === Feature Collection ===
print("Building features...")
# Initialize lists to collect features (X_all) and labels (y_all).
X_all, y_all = [], []

for year in selected_years:
    print(f"  Processing year {year}")
    # Load annual climate data:
    # - Sum precipitation over the year.
    # - Compute mean minimum and maximum temperatures.
    ppt = xr.open_dataset(os.path.join(ppt_dir, f"TerraClimate_ppt_{year}.nc"))['ppt'].sum(dim='time')
    tmin = xr.open_dataset(os.path.join(tmin_dir, f"TerraClimate_tmin_{year}.nc"))['tmin'].mean(dim='time')
    tmax = xr.open_dataset(os.path.join(tmax_dir, f"TerraClimate_tmax_{year}.nc"))['tmax'].mean(dim='time')

    # Get available latitude and longitude arrays from the precipitation dataset.
    lats = ppt.lat.values
    lons = ppt.lon.values

    # Extract occurrence records (presences) for the current year.
    presences = occurrences[occurrences['year'] == year]

    # Process each occurrence record.
    for _, row in presences.iterrows():
        lat, lon = row['latitude'], row['longitude']
        try:
            # Convert geographic coordinates to raster indices.
            i, j = rowcol(transform, lon, lat)
            # Skip this record if it does not fall on land.
            if not landmask[i, j]:
                continue
            elev_val = elev[i, j]
            # Extract climate features using nearest-neighbor selection.
            feat = [
                float(ppt.sel(lat=lat, lon=lon, method='nearest')),
                float(tmin.sel(lat=lat, lon=lon, method='nearest')),
                float(tmax.sel(lat=lat, lon=lon, method='nearest')),
                elev_val
            ]
            # Append the features and label (1 for presence) if all values are valid.
            if not any(np.isnan(feat)):
                X_all.append(feat)
                y_all.append(1)
        except Exception:
            continue

    # Generate pseudo-absence points randomly.
    absences = 0
    while absences < len(presences) * 2:
        # Randomly select a latitude and longitude from the available climate grid.
        rand_lat = float(np.random.choice(lats))
        rand_lon = float(np.random.choice(lons))
        try:
            i, j = rowcol(transform, rand_lon, rand_lat)
            # Ensure the randomly chosen point falls on land.
            if not landmask[i, j]:
                continue
            elev_val = elev[i, j]
            feat = [
                float(ppt.sel(lat=rand_lat, lon=rand_lon, method='nearest')),
                float(tmin.sel(lat=rand_lat, lon=rand_lon, method='nearest')),
                float(tmax.sel(lat=rand_lat, lon=rand_lon, method='nearest')),
                elev_val
            ]
            # Add the pseudo-absence record if it contains valid feature values.
            if not any(np.isnan(feat)):
                X_all.append(feat)
                y_all.append(0)
                absences += 1
        except Exception:
            continue

print(f"\nFeatures collected: {len(X_all)} samples")

# === Model Evaluation ===
# Convert the feature and label lists to NumPy arrays.
X = np.array(X_all)
y = np.array(y_all)

# Split the dataset into training and testing subsets.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Load the pre-trained Random Forest model.
with open("sdm_final/random_forest_model.pkl", "rb") as f:
    model = pickle.load(f)

# Make predictions on the test set.
y_probs = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

# Calculate the ROC AUC score to evaluate model performance.
auc = roc_auc_score(y_test, y_probs)
print(f"\nROC AUC Score: {auc:.3f}")

# Display the confusion matrix.
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Display the full classification report.
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Plot the ROC Curve using the estimator's performance on the test data.
RocCurveDisplay.from_estimator(model, X_test, y_test)
plt.title("ROC Curve")
plt.grid(True)
plt.show()

# Plot Feature Importance as a horizontal bar chart.
feature_names = ['Precipitation', 'Min Temp', 'Max Temp', 'Elevation']
importances = model.feature_importances_

plt.figure()
plt.barh(feature_names, importances, color='orange')
plt.xlabel("Importance")
plt.title("Feature Importances")
plt.grid(True)
plt.show()

# Print out the feature importances.
print("\nFeature Importances:")
for name, score in zip(feature_names, importances):
    print(f"  - {name}: {score:.3f}")


## 2.3) Prediction and Conclusions

In [ ]:
import os
import numpy as np
import pickle
import rasterio
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.ndimage import zoom, gaussian_filter

# Define file and directory paths for outputs, elevation, landmask, and climate data.
output_dir = "sdm_final"
elevation_path = "elevation_resampled_to_climate.tif"
landmask_path = "landmask_asia.tif"
climate_root = r"C:\Users\FENIL\Downloads\climate_data"
ppt_dir = os.path.join(climate_root, "ppt_1990_2020")
tmin_dir = os.path.join(climate_root, "tmin_1990_2020")
tmax_dir = os.path.join(climate_root, "tmax_1990_2020")
os.makedirs(output_dir, exist_ok=True)

# Load the pre-trained Random Forest model from disk.
with open(os.path.join(output_dir, "random_forest_model.pkl"), 'rb') as f:
    model = pickle.load(f)

# Open the elevation raster and read the first band.
with rasterio.open(elevation_path) as elev_src:
    elev = elev_src.read(1)

# Open the landmask raster and convert the data to boolean values.
with rasterio.open(landmask_path) as lm_src:
    landmask = lm_src.read(1).astype(bool)

# Function to load monthly climate bands and reduce them either by summing or by averaging.
def load_bands(path, reduce='sum'):
    ds = xr.open_dataset(path)
    bands = [ds[f'Band{i}'] for i in range(1, 13)]
    return sum(bands) if reduce == 'sum' else sum(bands) / 12

# Function to predict habitat suitability and generate a visualization map.
def predict_and_plot(year, suffix, label):
    print(f"\nProcessing predictions for year {year} ({label})")

    # Determine if the prediction is for a future scenario based on the suffix.
    is_future = isinstance(suffix, str) and suffix.startswith("2050")
    if is_future:
        # For future scenarios, load and process the climate data and reverse the vertical axis.
        ppt = load_bands(os.path.join(ppt_dir, f"TerraClimate_ppt_{suffix}.nc"), 'sum')[::-1, :]
        tmin = load_bands(os.path.join(tmin_dir, f"TerraClimate_tmin_{suffix}.nc"), 'mean')[::-1, :]
        tmax = load_bands(os.path.join(tmax_dir, f"TerraClimate_tmax_{suffix}.nc"), 'mean')[::-1, :]
    else:
        # For observed data, load the NetCDF file and compute the necessary aggregate values.
        ppt = xr.open_dataset(os.path.join(ppt_dir, f"TerraClimate_ppt_{year}.nc"))['ppt'].sum(dim='time')
        tmin = xr.open_dataset(os.path.join(tmin_dir, f"TerraClimate_tmin_{year}.nc"))['tmin'].mean(dim='time')
        tmax = xr.open_dataset(os.path.join(tmax_dir, f"TerraClimate_tmax_{year}.nc"))['tmax'].mean(dim='time')

    # Stack the climate and elevation variables into a 2D feature array.
    flat_features = np.stack([
        ppt.values.flatten(),
        tmin.values.flatten(),
        tmax.values.flatten(),
        elev.flatten()
    ], axis=1)

    # Create a mask for valid data locations (non-NaN values and on land).
    valid = (~np.isnan(flat_features).any(axis=1)) & landmask.flatten()

    # Prepare a full array to hold the prediction probabilities.
    y_pred = np.full(ppt.values.size, np.nan)
    # For valid data points, compute the probability of presence using the trained model.
    y_pred[valid] = model.predict_proba(flat_features[valid])[:, 1]

    # Reshape the predicted probabilities to match the original climate raster shape.
    suitability_map = y_pred.reshape(ppt.values.shape)
    # Assign NaN to locations not on land.
    suitability_map[~landmask] = np.nan

    # Save the raw suitability map as a NumPy binary file.
    np.save(os.path.join(output_dir, f"suitability_map_{year}_{label}.npy"), suitability_map)

    # Prepare a display map by filtering out low suitability values.
    display_map = suitability_map.copy()
    display_map[display_map <= 0.5] = np.nan
    # Increase the resolution of the map for better visualization.
    upscale = zoom(display_map, 3, order=1)
    # Apply a Gaussian filter to smooth the map.
    smooth_map = gaussian_filter(upscale, sigma=1.2)

    # Create a figure with the Plate Carree projection.
    fig = plt.figure(figsize=(12, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    # Set the spatial extent for the map display.
    ax.set_extent([40, 140, -5, 60], crs=ccrs.PlateCarree())
    # Add a background image and geographic features.
    ax.stock_img()
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.BORDERS, linewidth=0.4)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.4)

    # Display the smoothed habitat suitability map.
    img = ax.imshow(smooth_map, cmap='YlOrRd', alpha=0.4,
                    extent=[float(ppt.lon.min()), float(ppt.lon.max()), float(ppt.lat.min()), float(ppt.lat.max())],
                    transform=ccrs.PlateCarree(), vmin=0.5, vmax=1)

    # Add a horizontal colorbar with an appropriate label.
    cbar = plt.colorbar(img, orientation='horizontal', pad=0.05, shrink=0.75)
    cbar.set_label(f"Wild Yak Suitability ({label})")
    plt.title(f"Wild Yak Habitat Suitability - {year}", fontsize=14)

    # Add an arrow and label to indicate the North direction.
    ax.annotate('', xy=(0.94, 0.88), xytext=(0.94, 0.82),
                xycoords='axes fraction', arrowprops=dict(facecolor='black', arrowstyle='-|>', lw=1.5))
    ax.text(0.94, 0.89, 'N', transform=ax.transAxes,
            horizontalalignment='center', verticalalignment='bottom',
            fontsize=12, fontweight='bold', color='black')

    # Annotate country names on the map for reference.
    country_labels = {
        'India': ((78, 22), 9), 'China': ((104, 35), 10), 'Nepal': ((84, 28.5), 8),
        'Mongolia': ((103, 47), 9), 'Bangladesh': ((90, 24), 8), 'Pakistan': ((70, 30), 8),
        'Afghanistan': ((66, 34), 8), 'Bhutan': ((91.5, 27.5), 7),
        'Myanmar': ((96, 21), 8), 'Kazakhstan': ((70, 48), 9)
    }
    for name, ((lon, lat), fontsize) in country_labels.items():
        ax.text(lon, lat, name, transform=ccrs.PlateCarree(),
                fontsize=fontsize, fontweight='bold', ha='center', color='black',
                bbox=dict(facecolor='white', alpha=0.6, boxstyle='round,pad=0.2'))

    # Save the generated map as a high-resolution PNG file.
    save_path = os.path.join(output_dir, f"yak_suitability_{year}_{label}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Map and .npy file saved: {save_path}")

# Define years for which predictions will be generated.
selected_years = list(range(2009, 2025))
# Define future climate scenarios for the year 2050 with different SSP labels.
future_scenarios = {2050: {"SSP585": "2050585", "SSP245": "2050245"}}

# Generate predictions and plot maps for each selected observed year.
for year in selected_years:
    predict_and_plot(year, year, str(year))

# Generate predictions and plot maps for each future climate scenario.
for future_year, scenarios in future_scenarios.items():
    for label, suffix in scenarios.items():
        predict_and_plot(future_year, suffix, label)


In [ ]:
import numpy as np
import rasterio
import os

# Path to the input landmask raster file.
tif_path = "landmask_asia.tif"
# Path where the NumPy binary file (npy) will be saved.
output_path = "sdm_final/landmask.npy"

# Open the raster file and read the first band.
with rasterio.open(tif_path) as src:
    land_data = src.read(1)  # Read the first band of the raster.
    # Convert the raster data to a boolean mask where values greater than 0 indicate land.
    landmask = (land_data > 0)

# Save the boolean landmask array as a NumPy .npy file.
np.save(output_path, landmask)
print(f"Landmask saved to: {output_path}")
print(f"Shape: {landmask.shape}, Land pixels: {np.count_nonzero(landmask)}")


In [ ]:
import os
import numpy as np
import rasterio
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
output_dir = "sdm_final"
land_raster_path = os.path.join("landmask_asia.tif")
landmask_npy_path = os.path.join(output_dir, "landmask_asia_cropped.npy")
years = list(range(2009, 2025))
future = {
    "2050_SSP245": "suitability_map_2050_SSP245.npy",
    "2050_SSP585": "suitability_map_2050_SSP585.npy"
}
threshold = 0.5
pixel_area_km2 = 13.67  # Area per pixel (2.5 arcmin resolution)

# --- STEP 1: Load landmask raster and manually crop to Asia region ---
with rasterio.open(land_raster_path) as src:
    land_data = src.read(1)  # Read first band

# Manual crop to approximate Asia region (based on your raster grid: 4320 x 8640)
# Adjust these indices if needed — this focuses on central & eastern Asia
asia_mask = np.zeros_like(land_data, dtype=bool)
asia_mask[1000:3500, 2000:7000] = land_data[1000:3500, 2000:7000] > 0

# Save the cropped Asia landmask
np.save(landmask_npy_path, asia_mask)
print(f"✅ Saved cropped Asia landmask: {landmask_npy_path}")
print(f"Pixels in Asia mask: {np.count_nonzero(asia_mask)}")
print(f"Estimated total Asia area: {np.count_nonzero(asia_mask) * pixel_area_km2:,.2f} km²")

# --- STEP 2: Function to calculate suitable area in km² using Asia mask ---
def count_suitable_area_km2(npy_path, threshold, mask):
    data = np.load(npy_path)
    masked_data = np.where(mask, data, np.nan)
    suitable_pixels = np.count_nonzero(masked_data > threshold)
    return suitable_pixels * pixel_area_km2

# --- STEP 3: Collect suitable area for each year ---
area_by_year = []
for year in years:
    npy_file = os.path.join(output_dir, f"suitability_map_{year}_{year}.npy")
    if os.path.exists(npy_file):
        area = count_suitable_area_km2(npy_file, threshold, asia_mask)
        area_by_year.append((year, area))
    else:
        print(f"⚠️ Missing file: {npy_file}")

# --- STEP 4: Add future climate scenarios ---
for label, fname in future.items():
    npy_path = os.path.join(output_dir, fname)
    if os.path.exists(npy_path):
        area = count_suitable_area_km2(npy_path, threshold, asia_mask)
        area_by_year.append((label, area))
    else:
        print(f"⚠️ Missing file: {npy_path}")

# --- STEP 5: Plot the trend ---
area_by_year = sorted(area_by_year, key=lambda x: str(x[0]))
labels, areas_km2 = zip(*area_by_year)

plt.figure(figsize=(10, 6))
plt.plot(labels, areas_km2, marker='o', linewidth=2, color='seagreen', label='Suitable Area')

if '2050_SSP245' in labels:
    plt.axvline('2050_SSP245', color='orange', linestyle='--', label='SSP245 (2050)')
if '2050_SSP585' in labels:
    plt.axvline('2050_SSP585', color='red', linestyle='--', label='SSP585 (2050)')

plt.title(f"Suitable Habitat Area for Wild Yaks in Asia (Threshold > {threshold})", fontsize=14)
plt.ylabel("Suitable Area (km²)", fontsize=12)
plt.xlabel("Year / Scenario", fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()

plot_path = os.path.join(output_dir, "yak_suitability_area_trend_final.png")
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"📊 Plot saved to: {plot_path}")


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# --- Configuration ---
# Define the path to the CSV file containing centroid shifts.
csv_path = os.path.join("sdm_final", "centroid_shifts.csv")
# Define the output directory and file name for the trend plot.
output_dir = "sdm_final"
output_file = os.path.join(output_dir, "centroid_shift_trends.png")

# --- Load and Prepare Data ---
# Read the CSV file into a DataFrame.
df = pd.read_csv(csv_path)

# Create a numeric year field by extracting the numeric part from the 'Year' column.
# This handles entries with an underscore by splitting and converting the first part to an integer.
df["Year_Num"] = df["Year"].apply(lambda x: int(x.split("_")[0]) if "_" in x else int(x))
# Sort the DataFrame based on the numeric year.
df = df.sort_values("Year_Num")

# Create a mask to identify future scenarios (where the 'Year' string contains "2050").
future_mask = df["Year"].str.contains("2050", na=False)

# --- Plotting ---
# Create a figure with three vertically stacked subplots sharing the x-axis.
fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

# Plot the centroid latitude trend.
axs[0].plot(df["Year"], df["Centroid_Lat"], marker='o', color='royalblue')
# Highlight future scenarios with a scatter plot.
axs[0].scatter(df["Year"][future_mask], df["Centroid_Lat"][future_mask], color='black', label='2050 Scenarios')
axs[0].set_ylabel("Latitude (°N)")
axs[0].set_title("Centroid Latitude Shift")
axs[0].grid(True, linestyle='--', alpha=0.5)

# Plot the centroid longitude trend.
axs[1].plot(df["Year"], df["Centroid_Lon"], marker='o', color='forestgreen')
axs[1].scatter(df["Year"][future_mask], df["Centroid_Lon"][future_mask], color='black')
axs[1].set_ylabel("Longitude (°E)")
axs[1].set_title("Centroid Longitude Shift")
axs[1].grid(True, linestyle='--', alpha=0.5)

# Plot the median elevation trend.
axs[2].plot(df["Year"], df["Median_Elevation"], marker='o', color='firebrick')
axs[2].scatter(df["Year"][future_mask], df["Median_Elevation"][future_mask], color='black')
axs[2].set_ylabel("Elevation (m)")
axs[2].set_title("Median Elevation Shift")
axs[2].grid(True, linestyle='--', alpha=0.5)
axs[2].set_xlabel("Year / Scenario")

# Rotate the x-axis labels for better readability.
plt.xticks(rotation=45)
plt.tight_layout()

# Save the plot as a high-resolution PNG file.
plt.savefig(output_file, dpi=300, bbox_inches='tight')
plt.show()

print(f"Centroid trend plot saved: {output_file}")


In [ ]:
# Re-imports after kernel reset
import os
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# Load centroid data
df = pd.read_csv("sdm_final/centroid_shifts.csv")
df["Year_Num"] = df["Year"].apply(lambda x: int(x.split("_")[0]) if "_" in x else int(x))
df = df.sort_values("Year_Num").reset_index(drop=True)

# Calculate distance shifts between consecutive years
distances_km = [0]  # Start with 0 for the first year
for i in range(1, len(df)):
    prev = (df.loc[i - 1, "Centroid_Lat"], df.loc[i - 1, "Centroid_Lon"])
    curr = (df.loc[i, "Centroid_Lat"], df.loc[i, "Centroid_Lon"])
    distance = geodesic(prev, curr).kilometers
    distances_km.append(distance)

df["Distance_Change_km"] = distances_km

# Save to CSV
csv_path = os.path.join("sdm_final", "centroid_distance_changes.csv")
df.to_csv(csv_path, index=False)



In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import geodesic

# Load the centroid data from the CSV file.
df = pd.read_csv("sdm_final/centroid_shifts.csv")

# Compute distances between successive centroids.
# Start with 0 km for the first year as there is no previous location.
distances = [0.0]
for i in range(1, len(df)):
    # Get the latitude and longitude of the previous centroid.
    prev = (df.loc[i-1, "Centroid_Lat"], df.loc[i-1, "Centroid_Lon"])
    # Get the latitude and longitude of the current centroid.
    curr = (df.loc[i, "Centroid_Lat"], df.loc[i, "Centroid_Lon"])
    # Calculate the geodesic distance in kilometers between the two centroids.
    dist_km = geodesic(prev, curr).kilometers
    # Append the distance (rounded to 2 decimal places) to the distances list.
    distances.append(round(dist_km, 2))

# Set up a colormap to assign a unique color for each year in the dataset.
cmap = plt.cm.get_cmap("tab20", len(df))
colors = [cmap(i) for i in range(len(df))]
# Create labels for each point that include the year and the computed distance.
labels = [f"{year} ({dist} km)" for year, dist in zip(df["Year"], distances)]

# Create a scatter plot with arrows connecting centroids.
plt.figure(figsize=(10, 8))
for i in range(len(df)):
    # Plot the centroid as a scatter point.
    plt.scatter(df.loc[i, "Centroid_Lon"], df.loc[i, "Centroid_Lat"], color=colors[i], s=60, label=labels[i])
    # If there is a previous point, draw a line connecting the previous and current centroids.
    if i > 0:
        plt.plot([df.loc[i-1, "Centroid_Lon"], df.loc[i, "Centroid_Lon"]],
                 [df.loc[i-1, "Centroid_Lat"], df.loc[i, "Centroid_Lat"]],
                 color="black", linewidth=1)

# Set the title and labels for the plot.
plt.title("Wild Yak Habitat Centroid Shift with Distance (2009–2024 + 2050)", fontsize=14)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
# Add a legend that displays the year and the distance, placing it outside the plot area.
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5), title="Year (Distance)")
plt.grid(True)
plt.tight_layout()

# Save the final plot as a high-resolution PNG file.
final_path = os.path.join("sdm_final", "centroid_arrow_clean_map.png")
plt.savefig(final_path, dpi=300)
plt.show()


# 3) SDM for Takin

## 3.1) Data cleaning and EDA

In [ ]:
import pandas as pd
import re

# === Step 1: Load your CSV ===
input_file = "C:\\Users\\FENIL\\Downloads\\budorcas_taxicolor_occurrences.csv"
df = pd.read_csv(input_file)

# === Step 2: Extract year from date ===
def extract_year(date_str):
    if pd.isna(date_str):
        return None
    if '/' in date_str:
        date_str = date_str.split('/')[0]
    match = re.search(r'\b(19|20)\d{2}\b', date_str)
    if match:
        return int(match.group(0))
    return None

df['year'] = df['date'].apply(extract_year)

# === Step 3: Filter by year (only >= 2009) ===
df = df[df['year'] >= 2009]

# === Step 4: Filter by region (remove Europe and Americas) ===
# General bounds for Asia:
# - Latitude: 5 to 60
# - Longitude: 60 to 150
df = df[
    (df['latitude'] >= 5) & (df['latitude'] <= 60) &
    (df['longitude'] >= 60) & (df['longitude'] <= 150)
]

# Optional: Narrow further to Himalayan-Tibetan region
# Example bounds (customize if needed):
# Latitude: 25 to 40
# Longitude: 75 to 100
# df = df[
#     (df['latitude'] >= 25) & (df['latitude'] <= 40) &
#     (df['longitude'] >= 75) & (df['longitude'] <= 100)
# ]

# === Step 5: Drop the original date column (if needed) ===
df_cleaned = df.drop(columns=['date'])

# === Step 6: Save cleaned data ===
output_file = "takin_cleaned_asia_after2009.csv"
df_cleaned.to_csv(output_file, index=False)

print(f"Filtered data saved to: {output_file}")


In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import distance
from geopy import Point
import random

# === Step 1: Load the cleaned CSV ===
df = pd.read_csv("takin_cleaned_asia_after2009.csv")

# === Step 2: Function to generate a random point within 5 km ===
def jitter_point(lat, lon, max_km=5):
    # Random bearing (0 to 360°)
    bearing = random.uniform(0, 360)
    # Random distance (0 to max_km)
    dist = random.uniform(0, max_km)
    origin = Point(lat, lon)
    destination = distance(kilometers=dist).destination(origin, bearing)
    return destination.latitude, destination.longitude

# === Step 3: Create jittered points ===
jittered_data = []

for _, row in df.iterrows():
    for _ in range(5):  # 3 jittered points per original
        jitter_lat, jitter_lon = jitter_point(row['latitude'], row['longitude'])
        jittered_data.append({
            'scientificName': row['scientificName'],
            'latitude': jitter_lat,
            'longitude': jitter_lon,
            'source': row['source'],
            'year': row['year']
        })

# === Step 4: Combine original and jittered data ===
df_jittered = pd.DataFrame(jittered_data)
df_combined = pd.concat([df, df_jittered], ignore_index=True)

# === Step 5: Save to CSV ===
output_file = "takin_Final_cleaned.csv"
df_combined.to_csv(output_file, index=False)

print(f"Jittered dataset saved to: {output_file}")



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# === Step 1: Load the cleaned CSV ===
df = pd.read_csv("takin_Final_cleaned.csv")

# === Step 2: Create a world map ===
plt.figure(figsize=(15, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()  # Show the entire world

# === Step 3: Add geographic features ===
ax.add_feature(cfeature.LAND, edgecolor='black')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.LAKES, alpha=0.4)
ax.add_feature(cfeature.RIVERS)

# === Step 4: Plot occurrence points ===
ax.scatter(df['longitude'], df['latitude'],
           color='red', s=12, alpha=0.7,
           transform=ccrs.PlateCarree(),
           label='Takin Occurrence')

# === Step 5: Display map ===
plt.title("Global Distribution of Cleaned Takin Occurrence Points (Post-2009)")
plt.legend(loc='lower left')
plt.tight_layout()
plt.show()


## 3.2) Modeling and Evaluation

In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import rasterio
from rasterio.transform import rowcol
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import pickle

# ========= CONFIGURATION =========
# Define file paths for occurrence data, elevation and landmask rasters, and climate data directories.
occurrence_csv = "takin_Final_cleaned.csv"
elevation_path = "elevation_resampled_to_climate.tif"
landmask_path = "landmask_asia.tif"
climate_root = r"C:\Users\FENIL\Downloads\climate_data"
ppt_dir = os.path.join(climate_root, "ppt_1990_2020")
tmin_dir = os.path.join(climate_root, "tmin_1990_2020")
tmax_dir = os.path.join(climate_root, "tmax_1990_2020")
output_dir = "sdm_takin"
os.makedirs(output_dir, exist_ok=True)

# Define the years for processing and future scenario information.
selected_years = list(range(2009, 2025))
future_scenarios = {2050: {"SSP585": "2050585", "SSP245": "2050245"}}
np.random.seed(42)

# ========= LOAD OCCURRENCE DATA =========
print("Loading occurrence records...")
# Read the occurrence records, keeping only the required columns and dropping missing values.
occurrences = pd.read_csv(occurrence_csv)
occurrences = occurrences[['latitude', 'longitude', 'year']].dropna()
# Filter occurrence records to include only the selected years.
occurrences = occurrences[occurrences['year'].isin(selected_years)]

# ========= LOAD RASTER DATA =========
print("Opening elevation and landmask rasters...")
# Open the elevation raster and read the first band along with its spatial transform.
with rasterio.open(elevation_path) as elev_src:
    elev = elev_src.read(1)
    transform = elev_src.transform

# Open the landmask raster and convert the data into a boolean array.
with rasterio.open(landmask_path) as lm_src:
    landmask = lm_src.read(1).astype(bool)

# ========= FEATURE COLLECTION =========
# Initialize lists to store features and corresponding presence/absence labels.
X_all, y_all = [], []
for year in selected_years:
    print(f"Processing year {year}...")
    # Load climate data for precipitation (total annual sum),
    # and minimum and maximum temperatures (annual means) using xarray.
    ppt = xr.open_dataset(os.path.join(ppt_dir, f"TerraClimate_ppt_{year}.nc"))['ppt'].sum(dim='time')
    tmin = xr.open_dataset(os.path.join(tmin_dir, f"TerraClimate_tmin_{year}.nc"))['tmin'].mean(dim='time')
    tmax = xr.open_dataset(os.path.join(tmax_dir, f"TerraClimate_tmax_{year}.nc"))['tmax'].mean(dim='time')

    # Get occurrence records for the current year.
    presences = occurrences[occurrences['year'] == year]

    # Process each occurrence record (presence points).
    for _, row in presences.iterrows():
        lat, lon = row['latitude'], row['longitude']
        # Translate latitude and longitude into raster row and column indices.
        i, j = rowcol(transform, lon, lat)
        # Skip the point if it does not lie on land.
        if not landmask[i, j]:
            continue
        # Extract climate variables at the location and get the corresponding elevation.
        feat = [
            float(ppt.sel(lat=lat, lon=lon, method='nearest')),
            float(tmin.sel(lat=lat, lon=lon, method='nearest')),
            float(tmax.sel(lat=lat, lon=lon, method='nearest')),
            elev[i, j]
        ]
        # If all feature values are valid numbers, add the features and assign a label of 1 for presence.
        if not np.isnan(feat).any():
            X_all.append(feat)
            y_all.append(1)

    # Generate pseudo-absences by sampling random locations.
    absences, attempts = 0, 0
    lats, lons = ppt.lat.values, ppt.lon.values
    # Continue sampling until twice as many absence points as presences are collected,
    # or until a maximum number of attempts is reached.
    while absences < len(presences) * 2 and attempts < 10000:
        rand_lat, rand_lon = np.random.choice(lats), np.random.choice(lons)
        attempts += 1
        i, j = rowcol(transform, rand_lon, rand_lat)
        # Ensure the random point lies on land.
        if not landmask[i, j]:
            continue
        feat = [
            float(ppt.sel(lat=rand_lat, lon=rand_lon, method='nearest')),
            float(tmin.sel(lat=rand_lat, lon=rand_lon, method='nearest')),
            float(tmax.sel(lat=rand_lat, lon=rand_lon, method='nearest')),
            elev[i, j]
        ]
        # Add the feature if it is valid, and label it as absence (0).
        if not np.isnan(feat).any():
            X_all.append(feat)
            y_all.append(0)
            absences += 1

print("Features collected.")

# ========= TRAIN MODEL =========
print("Training model...")
# Convert the collected feature list and labels to NumPy arrays.
X, y = np.array(X_all), np.array(y_all)
# Split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Initialize a Random Forest classifier with a defined number of trees and train it.
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)
# Compute the AUC score on the test set to evaluate model performance.
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print(f"Model trained. AUC: {auc:.3f}")

# Save the trained model to a file using pickle.
model_filename = os.path.join(output_dir, "random_forest_model_takin.pkl")
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)
print(f"Model saved to {model_filename}")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, RocCurveDisplay, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# --- MODEL EVALUATION ---

# Generate model predictions on the test dataset.
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Compute and print the Area Under the ROC Curve (AUC) score.
print(f"\nROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

# --- CONFUSION MATRIX ---
# Compute the confusion matrix comparing actual and predicted labels.
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
# Draw a heatmap of the confusion matrix using a blue color palette.
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Absence', 'Presence'],
            yticklabels=['Absence', 'Presence'])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

# --- CLASSIFICATION REPORT ---
# Print the detailed classification report including precision, recall, f1-score, etc.
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Absence', 'Presence']))

# --- ROC CURVE PLOT ---
# Plot the ROC Curve to visualize the trade-off between true positive rate and false positive rate.
RocCurveDisplay.from_estimator(model, X_test, y_test)
plt.title("ROC Curve")
plt.show()

# --- FEATURE IMPORTANCE ---
# Retrieve the feature importance scores and sort them in descending order.
feature_names = ['Precipitation', 'Min Temperature', 'Max Temperature', 'Elevation']
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

# Plot the feature importances as a horizontal bar chart.
plt.figure(figsize=(6, 4))
sns.barplot(x=importances[indices], y=[feature_names[i] for i in indices])
plt.title("Feature Importances")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()


In [ ]:
import os
import numpy as np

# Define the output directory where the .npy files are stored.
output_dir = "sdm_takin"  # Change this if your directory is different.

# List all files with the .npy extension in the specified directory.
npy_files = [f for f in os.listdir(output_dir) if f.endswith(".npy")]
if not npy_files:
    print("No .npy files found in:", output_dir)
    exit()

print(f"Found {len(npy_files)} .npy files:")
for file in npy_files:
    print(" -", file)

# Inspect one .npy file; modify the index if you prefer a different file.
sample_file = os.path.join(output_dir, npy_files[0])
print("\nInspecting:", sample_file)

# Load the data from the selected .npy file.
data = np.load(sample_file)
print("Shape:", data.shape)

# Calculate and print basic statistics for the loaded data.
print("Statistics:")
print("   Min:", np.nanmin(data))
print("   Max:", np.nanmax(data))
print("   Mean:", np.nanmean(data))
print("   Number of NaNs:", np.isnan(data).sum())


In [ ]:
import xarray as xr
import rasterio

# === FILE PATHS ===
# Define file paths for the elevation raster, landmask, and climate data.
elevation_path = "elevation_resampled_to_climate.tif"
landmask_path = "landmask_asia.tif"
climate_root = r"C:\Users\FENIL\Downloads\climate_data"
ppt_path = f"{climate_root}\\ppt_1990_2020\\TerraClimate_ppt_2009.nc"
tmin_path = f"{climate_root}\\tmin_1990_2020\\TerraClimate_tmin_2009.nc"
tmax_path = f"{climate_root}\\tmax_1990_2020\\TerraClimate_tmax_2009.nc"

print("\nChecking raster alignment...\n")

# === CLIMATE DATA ===
# Load and aggregate the precipitation data by summing over the time dimension.
print("Precipitation (PPT):")
ppt = xr.open_dataset(ppt_path)['ppt'].sum(dim='time')
print(" - Shape:", ppt.shape)
print(" - Latitude range:", float(ppt.lat.min()), "to", float(ppt.lat.max()))
print(" - Longitude range:", float(ppt.lon.min()), "to", float(ppt.lon.max()))

# Load and aggregate the minimum temperature data by averaging over time.
print("\nMinimum Temperature (TMIN):")
tmin = xr.open_dataset(tmin_path)['tmin'].mean(dim='time')
print(" - Shape:", tmin.shape)

# Load and aggregate the maximum temperature data by averaging over time.
print("\nMaximum Temperature (TMAX):")
tmax = xr.open_dataset(tmax_path)['tmax'].mean(dim='time')
print(" - Shape:", tmax.shape)

# === ELEVATION ===
# Open the elevation raster and read the first band.
print("\nElevation Raster:")
with rasterio.open(elevation_path) as elev_src:
    elev = elev_src.read(1)
    print(" - Shape:", elev.shape)
    print(" - Bounds:", elev_src.bounds)
    print(" - Resolution:", elev_src.res)

# === LANDMASK ===
# Open the landmask raster and read the first band.
print("\nLandmask Raster:")
with rasterio.open(landmask_path) as lm_src:
    landmask = lm_src.read(1)
    print(" - Shape:", landmask.shape)
    print(" - Bounds:", lm_src.bounds)
    print(" - Resolution:", lm_src.res)

# === SHAPE CHECK ===
print("\nShape Match Check:")
# Check if the shapes of the climate rasters (ppt, tmin, tmax) are identical.
if ppt.shape == tmin.shape == tmax.shape:
    print("Climate rasters match in shape.")
else:
    print("Climate rasters have mismatched shapes.")

# Check if the elevation and landmask rasters have matching shapes.
if elev.shape == landmask.shape:
    print("Elevation and landmask shapes match.")
else:
    print("Elevation and landmask shapes differ.")


In [ ]:
selected_years = list(range(2019, 2025))
future_scenarios = {2050: {"SSP585": "2050585", "SSP245": "2050245"}}

for year in selected_years:
    predict_and_plot(year, year, str(year))

for future_year, scenarios in future_scenarios.items():
    for label, suffix in scenarios.items():
        predict_and_plot(future_year, suffix, label)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
# Define the output directory where the .npy files are stored.
output_dir = "sdm_takin"
# Area per pixel (in square kilometers) at 2.5 arcmin resolution.
pixel_area_km2 = 13.67
# Suitability threshold for habitat.
threshold = 0.5
# Define the years for the analysis.
years = list(range(2009, 2025))
# Define the future scenario files with scenario labels.
future = {
    "2050_SSP245": "suitability_map_2050_SSP245.npy",
    "2050_SSP585": "suitability_map_2050_SSP585.npy"
}

# --- STEP 1: Create Himalayan Mask using the correct lat/lon grid ---
# Load a sample suitability map to derive the grid dimensions.
sample_map_path = os.path.join(output_dir, "suitability_map_2024_2024.npy")
suitability_sample = np.load(sample_map_path)
lat_len, lon_len = suitability_sample.shape

# Create latitude values from 90 to -90 for the grid.
latitudes = np.linspace(90, -90, lat_len)
# Create longitude values from -180 to 180 (fixing the default from 0–360).
longitudes = np.linspace(-180, 180, lon_len)
# Create meshgrids for latitude and longitude.
lat_grid, lon_grid = np.meshgrid(latitudes, longitudes, indexing='ij')

# Define the Himalayan region based on latitude and longitude boundaries:
# Here, the region roughly covers India, Nepal, Bhutan, and southeastern Tibet.
himalaya_mask = (
    (lat_grid >= 25) & (lat_grid <= 38) &
    (lon_grid >= 78) & (lon_grid <= 105)
)

print(f"Mask shape: {himalaya_mask.shape}")
print(f"Pixels in Himalayan mask: {np.count_nonzero(himalaya_mask)}")
print(f"Estimated Himalayan area: {np.count_nonzero(himalaya_mask) * pixel_area_km2:,.2f} km²")

# --- STEP 2: Function to calculate suitable area using the mask ---
def count_suitable_area(npy_path, threshold, mask):
    """
    Load a suitability map from a .npy file, apply a mask, and count the area where suitability exceeds the threshold.
    
    Parameters:
        npy_path (str): File path for the .npy suitability map.
        threshold (float): Suitability threshold value.
        mask (np.ndarray): Boolean array defining the region of interest.
    
    Returns:
        float: Total suitable area in km².
    """
    data = np.load(npy_path)
    # Apply the mask to the data; pixels outside the mask become NaN.
    masked = np.where(mask, data, np.nan)
    # Count the number of pixels where suitability is above the threshold.
    suitable_pixels = np.count_nonzero(masked > threshold)
    return suitable_pixels * pixel_area_km2

# --- STEP 3: Calculate area for each year ---
area_by_year = []
for year in years:
    npy_file = os.path.join(output_dir, f"suitability_map_{year}_{year}.npy")
    if os.path.exists(npy_file):
        area = count_suitable_area(npy_file, threshold, himalaya_mask)
        area_by_year.append((year, area))
    else:
        print(f"Missing file: {npy_file}")

# --- STEP 4: Add future scenarios ---
for label, filename in future.items():
    npy_path = os.path.join(output_dir, filename)
    if os.path.exists(npy_path):
        area = count_suitable_area(npy_path, threshold, himalaya_mask)
        area_by_year.append((label, area))
    else:
        print(f"Missing file: {npy_path}")

# --- STEP 5: Plot the trend ---
# Sort the results so that labels are in logical order.
area_by_year = sorted(area_by_year, key=lambda x: str(x[0]))
labels, areas = zip(*area_by_year)

plt.figure(figsize=(10, 6))
plt.plot(labels, areas, marker='o', linewidth=2, color='darkgreen', label='Himalayan Suitable Area')

# Draw vertical lines to denote the future scenarios for clarity.
if '2050_SSP245' in labels:
    plt.axvline('2050_SSP245', color='orange', linestyle='--', label='SSP245 (2050)')
if '2050_SSP585' in labels:
    plt.axvline('2050_SSP585', color='red', linestyle='--', label='SSP585 (2050)')

plt.title("Takin Habitat Area Trend in the Himalayas (Threshold > 0.5)", fontsize=14)
plt.ylabel("Suitable Area (km²)", fontsize=12)
plt.xlabel("Year / Scenario", fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()

# --- STEP 6: Save and Show Plot ---
plot_path = os.path.join(output_dir, "takin_himalaya_area_trend.png")
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"Area trend plot saved to: {plot_path}")

# --- STEP 7: Optional - Print values year by year ---
print("\nYear-wise Habitat Area:")
for label, area in area_by_year:
    print(f"{label}: {area:,.2f} km²")


In [ ]:
import os
import numpy as np
import rasterio
import xarray as xr
import pandas as pd

# --- CONFIGURATION ---
# Specify directories and file paths for output, elevation, landmask, and climate data.
output_dir = "sdm_takin"
elevation_path = r"elevation_resampled_to_climate.tif"
landmask_path = r"landmask_asia.tif"
ppt_dir = r"C:\Users\FENIL\Downloads\climate_data\ppt_1990_2020"

# List of past years to process and definitions for future scenarios.
selected_years = list(range(2009, 2025))
future_scenarios = {
    "2050_SSP245": "SSP245",
    "2050_SSP585": "SSP585"
}
# Suitability threshold for defining habitat.
threshold = 0.5

# --- Load Elevation and Landmask ---
# Open the elevation raster and read the first band.
with rasterio.open(elevation_path) as elev_src:
    elev = elev_src.read(1)

# Open the landmask raster and convert it to a boolean array (True = land).
with rasterio.open(landmask_path) as lm_src:
    landmask = lm_src.read(1).astype(bool)

# --- Get Latitude/Longitude Grid from a Sample NetCDF File ---
# Open one of the climate NetCDF files to retrieve latitude and longitude values.
sample_nc = xr.open_dataset(os.path.join(ppt_dir, "TerraClimate_ppt_2019.nc"))
lat_vals = sample_nc['lat'].values
lon_vals = sample_nc['lon'].values
# Create a meshgrid that matches the raster grid's dimensions.
lat_grid, lon_grid = np.meshgrid(lat_vals, lon_vals, indexing='ij')

# --- Function to Calculate Centroid with Outlier Trimming ---
def compute_centroid(data, lat_grid, lon_grid, elev_map, threshold=0.5):
    """
    Compute the centroid of the region where suitability exceeds the threshold,
    trimming out outlier values based on percentiles.

    Parameters:
        data (ndarray): Suitability map.
        lat_grid (ndarray): Grid of latitude values.
        lon_grid (ndarray): Grid of longitude values.
        elev_map (ndarray): Elevation data aligned with the climate grid.
        threshold (float): Suitability threshold for habitat.

    Returns:
        tuple: (mean latitude, mean longitude, median elevation) of the region.
    """
    # Create a boolean mask for pixels above the suitability threshold.
    mask = data > threshold
    if not np.any(mask):
        return np.nan, np.nan, np.nan

    # Extract latitudes, longitudes, and elevation values of suitable pixels.
    lats = lat_grid[mask]
    lons = lon_grid[mask]
    elevs = elev_map[mask]

    # Determine the 5th and 95th percentile to trim outlier values.
    lat_low, lat_high = np.percentile(lats, [5, 95])
    lon_low, lon_high = np.percentile(lons, [5, 95])
    elev_low, elev_high = np.percentile(elevs, [5, 95])

    # Retain only those values that fall within the trimmed percentile range.
    valid = (lats >= lat_low) & (lats <= lat_high) & \
            (lons >= lon_low) & (lons <= lon_high) & \
            (elevs >= elev_low) & (elevs <= elev_high)

    # Compute and return the centroid (mean latitude and longitude) and median elevation.
    return np.mean(lats[valid]), np.mean(lons[valid]), np.median(elevs[valid])

# --- Collect Centroid Data for Each Scenario ---
centroid_data = []

# Loop over past years and compute centroids.
for year in selected_years:
    npy_path = os.path.join(output_dir, f"suitability_map_{year}_{year}.npy")
    if os.path.exists(npy_path):
        data = np.load(npy_path)
        lat_c, lon_c, elev_c = compute_centroid(data, lat_grid, lon_grid, elev, threshold)
        centroid_data.append((str(year), lat_c, lon_c, elev_c))
    else:
        print(f"File not found: {npy_path}")

# Loop over future scenarios and compute centroids.
for label, suffix in future_scenarios.items():
    npy_path = os.path.join(output_dir, f"suitability_map_2050_{suffix}.npy")
    if os.path.exists(npy_path):
        data = np.load(npy_path)
        lat_c, lon_c, elev_c = compute_centroid(data, lat_grid, lon_grid, elev, threshold)
        centroid_data.append((label, lat_c, lon_c, elev_c))
    else:
        print(f"File not found: {npy_path}")

# --- Save the Centroid Data to a CSV File ---
# Convert the centroid data to a DataFrame, sort by the 'Year' column, and reset the index.
centroid_df = pd.DataFrame(centroid_data, columns=["Year", "Centroid_Lat", "Centroid_Lon", "Median_Elevation"])
centroid_df.sort_values("Year", inplace=True)
centroid_df.reset_index(drop=True, inplace=True)

# Define the CSV output path and save the DataFrame.
csv_path = os.path.join(output_dir, "centroid_shifts_takin.csv")
centroid_df.to_csv(csv_path, index=False)

print(f"\nTakin centroid data saved to: {csv_path}")
print(centroid_df)


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
# Define the path to the CSV file containing centroid shift data.
csv_path = os.path.join("sdm_takin", "centroid_shifts_takin.csv")
# Define the output directory for saving the plot.
output_dir = "sdm_takin"
# Specify the file name and path for the trend plot.
output_file = os.path.join(output_dir, "centroid_shift_trends_takin.png")

# --- LOAD AND PREPARE DATA ---
# Read the centroid shifts CSV file into a DataFrame.
df = pd.read_csv(csv_path)
# Create a numeric year column for proper sorting.
# If the 'Year' value contains an underscore, split it and use the first part; otherwise, convert directly.
df["Year_Num"] = df["Year"].apply(lambda x: int(x.split("_")[0]) if "_" in x else int(x))
# Sort the DataFrame based on the numeric year.
df = df.sort_values("Year_Num")

# Create a mask to identify the future scenarios (those with '2050' in the Year field).
future_mask = df["Year"].str.contains("2050", na=False)

# --- PLOT THE TRENDS ---
# Set up a figure with three subplots for latitude, longitude, and elevation.
fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

# Plot centroid latitude shifts.
axs[0].plot(df["Year"], df["Centroid_Lat"], marker='o', color='royalblue')
# Highlight the 2050 scenario points with a different color.
axs[0].scatter(df["Year"][future_mask], df["Centroid_Lat"][future_mask], color='black', label='2050 Scenarios')
axs[0].set_ylabel("Latitude (°N)")
axs[0].set_title("Centroid Latitude Shift")
axs[0].grid(True, linestyle='--', alpha=0.5)

# Plot centroid longitude shifts.
axs[1].plot(df["Year"], df["Centroid_Lon"], marker='o', color='forestgreen')
axs[1].scatter(df["Year"][future_mask], df["Centroid_Lon"][future_mask], color='black')
axs[1].set_ylabel("Longitude (°E)")
axs[1].set_title("Centroid Longitude Shift")
axs[1].grid(True, linestyle='--', alpha=0.5)

# Plot median elevation shifts.
axs[2].plot(df["Year"], df["Median_Elevation"], marker='o', color='firebrick')
axs[2].scatter(df["Year"][future_mask], df["Median_Elevation"][future_mask], color='black')
axs[2].set_ylabel("Elevation (m)")
axs[2].set_title("Median Elevation Shift")
axs[2].grid(True, linestyle='--', alpha=0.5)
axs[2].set_xlabel("Year / Scenario")

# Rotate x-axis labels for better readability and adjust layout.
plt.xticks(rotation=45)
plt.tight_layout()

# Save the resulting plot as a high-resolution PNG file.
plt.savefig(output_file, dpi=300, bbox_inches='tight')
plt.show()

# Inform the user that the plot has been saved.
print(f"Takin centroid trend plot saved: {output_file}")


In [ ]:
import os
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# --- Load the centroid shift CSV for Takin ---
# Read the centroid shifts data which includes year, latitude, and longitude information.
df = pd.read_csv("sdm_takin/centroid_shifts_takin.csv")

# --- Sort by Numeric Year ---
# Create a numeric representation of the Year column to sort the data chronologically.
df["Year_Num"] = df["Year"].apply(lambda x: int(x.split("_")[0]) if "_" in x else int(x))
df = df.sort_values("Year_Num").reset_index(drop=True)

# --- Calculate Geodesic Distances Between Consecutive Years ---
# Initialize the list of distances. The first year has no previous record, so its shift is 0 km.
distances_km = [0]
# Loop over each record (starting from the second) and compute the geodesic distance from the previous year's centroid.
for i in range(1, len(df)):
    prev = (df.loc[i - 1, "Centroid_Lat"], df.loc[i - 1, "Centroid_Lon"])
    curr = (df.loc[i, "Centroid_Lat"], df.loc[i, "Centroid_Lon"])
    # Calculate the distance in kilometers using geopy's geodesic method.
    distance = geodesic(prev, curr).kilometers
    distances_km.append(distance)

# Add the calculated distances as a new column in the DataFrame.
df["Distance_Change_km"] = distances_km

# --- Save the Updated DataFrame ---
# Define the output CSV file path.
output_path = os.path.join("sdm_takin", "centroid_distance_changes_takin.csv")
# Write the updated DataFrame to the CSV file without the index.
df.to_csv(output_path, index=False)

# Print a confirmation message along with key columns to verify the results.
print(f"Distance shift table saved: {output_path}")
print(df[["Year", "Centroid_Lat", "Centroid_Lon", "Distance_Change_km"]])


In [ ]:
import os
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# --- Load the centroid shift CSV for Takin ---
# Read the CSV file that contains the centroid shift data (year, latitude, longitude).
df = pd.read_csv("sdm_takin/centroid_shifts_takin.csv")

# --- Sort by Numeric Year ---
# Create a numeric version of the "Year" column. If the year contains an underscore (e.g. for future scenarios),
# split the string and use the numeric part only. Otherwise, convert directly to integer.
df["Year_Num"] = df["Year"].apply(lambda x: int(x.split("_")[0]) if "_" in x else int(x))
# Sort the DataFrame based on the numeric year and reset the index.
df = df.sort_values("Year_Num").reset_index(drop=True)

# --- Calculate Geodesic Distances Between Consecutive Years ---
# Initialize a list to store the distance changes (in km). The first year is set to 0 as a starting point.
distances_km = [0]
# Loop through the DataFrame starting from the second row and calculate the geodesic distance between consecutive centroids.
for i in range(1, len(df)):
    # Get the centroid coordinates (latitude, longitude) for the previous year.
    prev = (df.loc[i - 1, "Centroid_Lat"], df.loc[i - 1, "Centroid_Lon"])
    # Get the centroid coordinates for the current year.
    curr = (df.loc[i, "Centroid_Lat"], df.loc[i, "Centroid_Lon"])
    # Calculate the geodesic distance between these two centroids in kilometers.
    distance = geodesic(prev, curr).kilometers
    distances_km.append(distance)

# Add the list of calculated distance changes as a new column in the DataFrame.
df["Distance_Change_km"] = distances_km

# --- Save the Updated DataFrame ---
# Define the output file path for saving the updated data.
output_path = os.path.join("sdm_takin", "centroid_distance_changes_takin.csv")
# Save the DataFrame to a CSV file without including the index.
df.to_csv(output_path, index=False)

# Print a confirmation message along with selected columns to verify the results.
print(f"Distance shift table saved: {output_path}")
print(df[["Year", "Centroid_Lat", "Centroid_Lon", "Distance_Change_km"]])
